In [153]:
import pandas as pd
import random
from snorkel.labeling import labeling_function

In [ ]:
# load the data by choosing a random 10% of the data
# is the data ordered? am I missing a large chunk due to my partial loading?
p = 0.01
responses = pd.read_csv('../data/moralmachine/SharedResponses.csv', skiprows=lambda i: i>0 and random.random() > p)
responses.shape

In [168]:
responses.describe()

,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,DefaultChoiceIsOmission,NumberOfCharacters,DiffNumberOFCharacters,Saved,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
count,7.022190e+05,702306.000000,702306.000000,702306.000000,702306.000000,702306.000000,627465.000000,702306.000000,702306.000000,702306.000000,...,702306.000000,702306.000000,702306.000000,702306.000000,702306.000000,702306.000000,702306.000000,702306.000000,702306.000000,702306.000000
mean,3.167897e+15,6.725433,0.499653,0.448686,0.276412,0.604442,0.485176,3.078425,0.568453,0.503876,...,0.159090,0.059319,0.106875,0.107171,0.190611,0.191345,0.096542,0.096330,0.172621,0.173427
std,3.332668e+15,3.773307,0.500000,0.497360,0.447223,0.814391,0.499781,1.462537,1.147207,0.499985,...,0.446436,0.254915,0.349162,0.349084,0.521901,0.521665,0.332641,0.331981,0.565638,0.567977
min,5.305100e+04,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.898494e+09,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.092663e+15,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.063048e+15,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.999993e+15,13.000000,1.000000,1.000000,1.000000,2.000000,1.000000,5.000000,4.000000,1.000000,...,5.000000,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.000000,5.000000,5.000000


In [172]:
## For viewing the possible values for each feature
for feature in responses.columns:
    print(feature)
    print(responses[feature].unique())
    print(responses[feature].unique().shape)

ResponseID
['222feB2SwrK38C5be' '222nXvJbgskDwC9AQ' '223Ey8FWWJSeCHRpW' ...
 'zzzShFoEBceAmrXWj' 'zzzie7NmtBx5KKb2w' 'zzznqmAu2uE9SYm3d']
(699015,)
ExtendedSessionID
['546506750_9988624123932244.0' '-1332809300_8398486853060885.0'
 '242352712_3528420733540.0' ... '1615899623_514570710'
 '-1254641538_4417535393209049.0' '397225291_9465514998451634.0']
(629987,)
UserID
[9.98862412e+15 8.39848685e+15 3.52842073e+12 ... 5.14570710e+08
 4.41753539e+15 9.46551500e+15]
(481815,)
ScenarioOrder
[ 6  3 10  2  7 11  9  4  5 12 13  1  8]
(13,)
Intervention
[0 1]
(2,)
PedPed
[1 0]
(2,)
Barrier
[0 1]
(2,)
CrossingSignal
[0 2 1]
(3,)
AttributeLevel
['Rand' 'Fit' 'Low' 'Female' 'Fat' 'Male' 'Pets' 'More' 'Young' 'Old'
 'Hoomans' 'Less' 'High']
(13,)
ScenarioTypeStrict
['Fitness' 'Social Status' 'Gender' 'Random' 'Species' 'Utilitarian' 'Age']
(7,)
ScenarioType
['Random' 'Fitness' 'Social Status' 'Gender' 'Species' 'Utilitarian' 'Age']
(7,)
DefaultChoice
[nan 'Fit' 'High' 'Male' 'Hoomans' 'More' 'Young

In [170]:
## What does a single user's session/response look like?
responses_random = responses[responses['ScenarioTypeStrict'] == 'Random']
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    ## see a random user's session
#     display(responses_random[responses_random['ExtendedSessionID'] == responses_random['ExtendedSessionID'].sample().values[0]])
    
    ## see the user with nth most responses
    n = 100
    display(responses_random[responses_random['ResponseID'] == responses_random.groupby(by='ResponseID').size().sort_values(ascending=False).index[n]])


,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,DefaultChoice,NonDefaultChoice,DefaultChoiceIsOmission,NumberOfCharacters,DiffNumberOFCharacters,Saved,Template,DescriptionShown,LeftHand,UserCountry3,Man,Woman,Pregnant,Stroller,OldMan,OldWoman,Boy,Girl,Homeless,LargeWoman,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
25645,5zZq9CN2YzFPFAJnA,1897907220_1899525873,1.899526e+09,10,0,0,1,0,Rand,Random,Random,NaN,NaN,NaN,2,1,0,Desktop,0.0,1.0,CAN,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
114298,5zZq9CN2YzFPFAJnA,1897907220_1899525873,1.899526e+09,10,1,0,0,2,Rand,Random,Random,NaN,NaN,NaN,3,1,1,Desktop,0.0,0.0,CAN,0.0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0


In [190]:
## group alternatives pairwise!
# take only responses where both alternatives are present in the random sample - for a full sample, this step isn't necessary
responses_grouped = responses.groupby(by='ResponseID').filter(lambda g: g.shape[0] < 2)
responses_grouped.shape

(695724, 41)

## Snorkel Labeling